In [2]:
import pandas as pd

In [4]:
#변형할 파일 submission으로 읽어오기
submission = pd.read_csv('./data/sample_submission.csv')
# submission = pd.read_csv('./submit/넣을파일.csv')

#isbn cold start
train = pd.read_csv('./data/train_ratings.csv')                 #train 값 불러오고

count=train.groupby("isbn").size()                              #isbn별로 묶어서 count
dfcount = pd.DataFrame(count, columns=["count_b"])
train=pd.merge(train,dfcount, how='left', on='isbn')            

submission=pd.merge(submission,dfcount, how='left', on='isbn')  #submission 에 count 생성
submission['count_b'] = submission['count_b'].fillna(0)


submission.set_index("isbn",inplace = True)                                
for row in submission.itertuples():
    if row[3] == 0 :                                            #book count가 되지 않은 곳에
        submission.at[row[0],"rating"] = 6.884027966331795      #1개 매겨진 isbn 의 평균 대입
submission = submission.reset_index()

#user_id cold start
train = pd.read_csv('./data/train_ratings.csv')

count=train.groupby("user_id").size()
dfcount = pd.DataFrame(count, columns=["count_u"])
train=pd.merge(train,dfcount, how='left', on='user_id')

submission=pd.merge(submission,dfcount, how='left', on='user_id')
submission['count_u'] = submission['count_u'].fillna(0)

submission.set_index("user_id",inplace = True)                              
for row in submission.itertuples():
    if row[4] == 0 :                                                       
        submission.at[row[0],"rating"] = 7                       #1명 rating 들의 평균대입
submission = submission.reset_index()


# #count_b, count_u 콜럼 제거
# submission = submission.drop(['count_b'], axis=1)
# submission = submission.drop(['count_u'], axis=1)

In [8]:
submission['count_u'].value_counts(sort= True)

0.0      8677
1.0      3860
2.0      3070
3.0      2480
4.0      2219
         ... 
165.0      37
155.0      36
151.0      30
131.0      29
89.0       20
Name: count_u, Length: 247, dtype: int64

In [34]:
user_cold = submission[submission['count_u'] ==0]
book_cold = submission[submission['count_b'] ==0]
len(book_cold)

19793

In [27]:
both_cold= book_cold[book_cold['count_u'].isin(user_cold['count_u'])]
len(both_cold)

1734

In [29]:
print("both test dataset size : ", len(both_cold))
print("cold start user size: ", len(user_cold))
print("cold start book size: ", len(book_cold))


both test dataset size :  1734
cold start user size:  8677
cold start book size:  19793


In [35]:
submission[submission['count_u'] ==0]
submission['count_u'] ==0
submission['count_b'] ==0

0        False
1        False
2        False
3        False
4        False
         ...  
76694    False
76695    False
76696    False
76697    False
76698    False
Name: count_u, Length: 76699, dtype: bool